In [22]:
%%time

import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
import scipy.stats as st
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split,KFold,StratifiedKFold,GridSearchCV,RandomizedSearchCV,cross_val_score
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import r2_score,roc_auc_score,recall_score,classification_report,mean_squared_error,accuracy_score,confusion_matrix

#for getting the columns only
test=pd.read_csv("cleaned_data_test_feature_engineered.csv")


#loading the data
data_train_smote=pd.read_csv("full_train_after_smote_adasyn.csv")
data_test_smote=pd.read_csv("cleaned_data_test_feature_engineered.csv")


#dropping the feature engineered data from test

data_test_smote=data_test_smote.drop(["YEAR","INCIDENT_ID","MONTH","DAY","DATE"],axis=1)
data_train_smote=data_train_smote.drop(["INCIDENT_ID"],axis=1)

print(data_train_smote.shape,data_test_smote.shape)


# #splitting into train and validation
# train,val=train_test_split(data_train_smote,test_size=.20,random_state=42)
# train.shape,val.shape


# #splitting into x and y
# Xfull=data_train_smote.drop(["MULTIPLE_OFFENSE"],axis=1)
# yfull=data_train_smote["MULTIPLE_OFFENSE"]
# print(Xfull.shape,yfull.shape)
# print(Xfull.shape,data_test_smote.shape)


#marker for concatenation of data
data_train_smote["mark"]="train"
data_test_smote["mark"]="test"

full_test_train_data=pd.concat([data_train_smote,data_test_smote])


#check for the columns to exclude 
def feature_enginnering_a_into_2b(colnames,Xfull,number_of_columns,flag=1):
    for column in colnames:
        if (flag<number_of_columns):
            print(column,colnames[flag])
            a=column+"into"+colnames[flag]
            Xfull[a]=Xfull[column]*(2*Xfull[colnames[flag]])
            print(a)
            flag+=1
        else:
            flag+=1
            print("exceeded")
    return Xfull


#calling feature engineering function
colnames=full_test_train_data.columns.values
colnames
p=len(colnames)
feature_engineered_dataframe_a_into_2b=feature_enginnering_a_into_2b(colnames=colnames,
                                                                    Xfull=full_test_train_data,
                                                                     number_of_columns=15)


print(feature_engineered_dataframe_a_into_2b.shape)

#sapretaing test and train data 
train_saperated=feature_engineered_dataframe_a_into_2b[feature_engineered_dataframe_a_into_2b["mark"]=="train"]
test_saperated=feature_engineered_dataframe_a_into_2b[feature_engineered_dataframe_a_into_2b["mark"]=="test"]
print(train_saperated.shape,test_saperated.shape)

#remocving unwanted columns
test_saperated.drop(["MULTIPLE_OFFENSE","MULTIPLE_OFFENSEintoX_1","mark"],axis=1,inplace=True)
train_saperated.drop(["MULTIPLE_OFFENSEintoX_1","mark"],axis=1,inplace=True)
print(train_saperated.shape,test_saperated.shape)


#splitting into train and validation
train,val=train_test_split(train_saperated,test_size=.20,random_state=42)
print(train.shape,val.shape)



#splitting into x and y
Xfull=train_saperated.drop(["MULTIPLE_OFFENSE"],axis=1)
yfull=train_saperated["MULTIPLE_OFFENSE"]
print(Xfull.shape,yfull.shape)


Xtrain=train.drop(["MULTIPLE_OFFENSE"],axis=1)
ytrain=train["MULTIPLE_OFFENSE"]
print(Xtrain.shape,ytrain.shape)

Xval=val.drop(["MULTIPLE_OFFENSE"],axis=1)
yval=val["MULTIPLE_OFFENSE"]
print(Xval.shape,yval.shape)


(45565, 16) (15903, 15)
MULTIPLE_OFFENSE X_1
MULTIPLE_OFFENSEintoX_1
X_1 X_10
X_1intoX_10
X_10 X_11
X_10intoX_11
X_11 X_12
X_11intoX_12
X_12 X_13
X_12intoX_13
X_13 X_14
X_13intoX_14
X_14 X_15
X_14intoX_15
X_15 X_2
X_15intoX_2
X_2 X_3
X_2intoX_3
X_3 X_4
X_3intoX_4
X_4 X_5
X_4intoX_5
X_5 X_6
X_5intoX_6
X_6 X_7
X_6intoX_7
X_7 X_8
X_7intoX_8
exceeded
exceeded
exceeded
(61468, 31)
(45565, 31) (15903, 31)
(45565, 29) (15903, 28)
Wall time: 1.67 s


In [24]:
lrc = LogisticRegression(class_weight="balanced")
rfc=RandomForestClassifier(class_weight="balanced")
adbc=AdaBoostClassifier()
bgc=BaggingClassifier()
gnb = GaussianNB()
knn=KNeighborsClassifier()
dtc = DecisionTreeClassifier()
bgcl_lrc = BaggingClassifier(base_estimator=lrc, random_state=42)
ab_rfc = AdaBoostClassifier(base_estimator=rfc,random_state=42)
ab_dtc = AdaBoostClassifier(base_estimator=dtc,random_state=42)
ab_nbc=  AdaBoostClassifier(base_estimator=gnb,random_state=42)
ab_lrc=  AdaBoostClassifier(base_estimator=lrc,random_state=42)
gbc=GradientBoostingClassifier()
ab_gbc=  AdaBoostClassifier(base_estimator=gbc,random_state=42)
xgbc=XGBClassifier()
ab_xgbc=  AdaBoostClassifier(base_estimator=xgbc,random_state=42)
lgbc=LGBMClassifier(class_weight="balanced")

In [25]:


models=[lrc,rfc,adbc,bgc,gnb,knn,dtc,bgcl_lrc,ab_rfc,ab_dtc,ab_nbc,ab_lrc,gbc,ab_gbc,xgbc,ab_xgbc,lgbc]
sctr,scte,auc,ps,rs=[],[],[],[],[]
def ensemble_for_train_val(X_train,X_test, y_train, y_test):
    for model in models:
            model.fit(X_train, y_train)
            y_test_pred = model.predict(X_test)
            y_test_pred_new=model.predict_proba(X_test)
            y_test_pred_new=y_test_pred_new[:,1]
            train_score=model.score(X_train,y_train)
            test_score=model.score(X_test,y_test)
            p_score=metrics.precision_score(y_test,y_test_pred)
            r_score=metrics.recall_score(y_test,y_test_pred)
            
            ac=metrics.roc_auc_score(y_test,y_test_pred_new)
            
            sctr.append(train_score)
            scte.append(test_score)
            ps.append(p_score)
            rs.append(r_score)
            auc.append(ac)
    return sctr,scte,auc,ps,rs


#checking the performance on train and validation
ensemble_for_train_val(Xtrain,Xval, ytrain, yval)
# 'ab_rf','ab_dt','ab_nb','ab_lr','bgcl_lr'
ensemble=pd.DataFrame({'names':['Logistic Regression','Random Forest','Ada boost','Bagging',
                                'Naive-Bayes','KNN','Decistion Tree',
                                'bagged LR',"adaboost rf","adaboost dtc","adaboost naive bayes",
                                "adaboost logistic regression","gradient boosting trees"
                                ,"adaboost gbc","xgboost","adaboost xgbc","ligh gbm"],
                       'auc_score':auc,'training':sctr,'testing':scte,'precision':ps,'recall':rs})
ensemble=ensemble.sort_values(by='auc_score',ascending=False).reset_index(drop=True)
ensemble

,names,auc_score,training,testing,precision,recall
0,adaboost gbc,0.999976,1.000000,0.999232,0.999119,0.999339
1,ligh gbm,0.999962,1.000000,0.998354,0.998678,0.998018
2,Random Forest,0.999926,1.000000,0.996379,0.999114,0.993614
3,adaboost rf,0.999822,1.000000,0.996269,0.999335,0.993173
4,xgboost,0.999790,0.995775,0.994623,0.995368,0.993834
5,gradient boosting trees,0.999760,0.996159,0.995391,0.996250,0.994495
6,Ada boost,0.999552,0.994102,0.994184,0.995583,0.992733
7,Bagging,0.998852,0.999534,0.996159,0.997132,0.995155
8,adaboost dtc,0.995498,1.000000,0.995501,0.996251,0.994715
9,Decistion Tree,0.995388,1.000000,0.995391,0.996250,0.994495


In [26]:
ensemble=ensemble.sort_values(by='recall',ascending=False).reset_index(drop=True)
ensemble

,names,auc_score,training,testing,precision,recall
0,adaboost xgbc,0.500000,0.500521,0.498299,0.498299,1.000000
1,adaboost gbc,0.999976,1.000000,0.999232,0.999119,0.999339
2,ligh gbm,0.999962,1.000000,0.998354,0.998678,0.998018
3,Bagging,0.998852,0.999534,0.996159,0.997132,0.995155
4,adaboost dtc,0.995498,1.000000,0.995501,0.996251,0.994715
5,gradient boosting trees,0.999760,0.996159,0.995391,0.996250,0.994495
6,Decistion Tree,0.995388,1.000000,0.995391,0.996250,0.994495
7,xgboost,0.999790,0.995775,0.994623,0.995368,0.993834
8,Random Forest,0.999926,1.000000,0.996379,0.999114,0.993614
9,adaboost rf,0.999822,1.000000,0.996269,0.999335,0.993173


In [27]:

def cross_val(fullx,fully,kfold,models,model_names,i=1,p=1):
    dictionary_cv={}
    cross_val_score_auc=[]
    for model in models:
        print(model_names[i-1])
        for train_index,test_index in kfold.split(fullx,fully):
            
            print('\n{} of kfold {}'.format(p,kf.n_splits))
            xtr,xvl = Xfull.loc[train_index],Xfull.loc[test_index]
            ytr,yvl = yfull[train_index],yfull[test_index]
            model.fit(xtr, ytr)
            pred=model.predict(xvl)
            print('roc_auc_score',roc_auc_score(yvl,pred))
            cross_val_score_auc.append(roc_auc_score(yvl,pred))
            p+=1
        temp_name=model_names[i-1]
        dictionary_cv[temp_name]=cross_val_score_auc
        cross_val_score_auc=[]
        p=1
        i+=1
    result_cross_validation=pd.DataFrame(list(dictionary_cv.items()))
    return result_cross_validation

kf = StratifiedKFold(n_splits=5,random_state=42,shuffle=True)

# models_to_cross_validate=[lrc,rfc]
# # models=
# model_names=['Logistic Regression','Random Forest',]

models_to_cross_validate=[lrc,rfc,adbc,bgc,gnb,knn,dtc,bgcl_lrc,
                          ab_rfc,ab_dtc,ab_nbc,ab_lrc,gbc,ab_gbc,xgbc,ab_xgbc,lgbc]
# models=
model_names=['Logistic_Regression','Random_Forest','Ada_boost','Bagging',
                                'Naive-Bayes','KNN','Decistion_Tree',
                                'bagged_LR',"adaboost_rf","adaboost_dtc","adaboost_naive_bayes",
                                "adaboost_logistic_regression","gradient_boosting_trees",
             "adaboost_gbc","xgboost","adaboost_xgbc","light_gbm"]

result_cv=cross_val(fullx=Xfull,fully=yfull,kfold=kf,
                    models=models_to_cross_validate,model_names=model_names)
result_cv.to_csv("cross_val_results_with_adasyn_withfeaturenegineer_a_into_2b.csv",index=False)

Logistic_Regression

1 of kfold 5
roc_auc_score 0.8578982869026281

2 of kfold 5
roc_auc_score 0.8592153279222137

3 of kfold 5
roc_auc_score 0.8589957889751954

4 of kfold 5
roc_auc_score 0.8608611957060147

5 of kfold 5
roc_auc_score 0.8588918580327517
Random_Forest

1 of kfold 5
roc_auc_score 0.9974763135875439

2 of kfold 5
roc_auc_score 0.9962690902070988

3 of kfold 5
roc_auc_score 0.9979151988190557

4 of kfold 5
roc_auc_score 0.9971471255810943

5 of kfold 5
roc_auc_score 0.9959406965425263
Ada_boost

1 of kfold 5
roc_auc_score 0.9937452592977489

2 of kfold 5
roc_auc_score 0.9934159267944057

3 of kfold 5
roc_auc_score 0.9944036112278324

4 of kfold 5
roc_auc_score 0.9940745677182765

5 of kfold 5
roc_auc_score 0.9915507841606344
Bagging

1 of kfold 5
roc_auc_score 0.9969275866340759

2 of kfold 5
roc_auc_score 0.995720291005184

3 of kfold 5
roc_auc_score 0.9969275384684446

4 of kfold 5
roc_auc_score 0.9970372116106914

5 of kfold 5
roc_auc_score 0.9948429294532382
Naive-Bay

In [33]:
# #splitting into x and y
# Xfull=train_saperated.drop(["MULTIPLE_OFFENSE"],axis=1)
# yfull=train_saperated["MULTIPLE_OFFENSE"]
# print(Xfull.shape,yfull.shape)
data_test_smote.columns

Index(['X_1', 'X_2', 'X_3', 'X_4', 'X_5', 'X_6', 'X_7', 'X_8', 'X_9', 'X_10',
       'X_11', 'X_12', 'X_13', 'X_14', 'X_15', 'mark'],
      dtype='object')

In [ ]:

# predictions=pd.DataFrame(data_test_smote['INCIDENT_ID'])
def pred_on_full_data(Xtrain,ytrain,Xtest,models,models_name,i=0):
    for model in models:
        print(models_name[i])
        model.fit(Xtrain, ytrain)
        y_test_pred = model.predict(Xtest)
        predictions=pd.concat([test['INCIDENT_ID'],pd.DataFrame(y_test_pred,columns=['MULTIPLE_OFFENSE'])],1)
        a="submission_feature_engineered"+models_name[i]+"with_adasync"+".csv"
        predictions.to_csv(a,index=False)
        i+=1

#models=[lrc,rfc,adbc,bgc,gnb,knn,dtc,bgcl_lrc,gbc,xgbc,lgbc]
# ab_rfc,ab_dtc,ab_nbc,ab_lrc,gbc,ab_gbc,xgbc,ab_xgbc
models_final_for_csv=[lrc,rfc,adbc,bgc,gnb,knn,dtc,bgcl_lrc,
                          ab_rfc,ab_dtc,ab_nbc,ab_lrc,gbc,ab_gbc,xgbc,ab_xgbc,lgbc]
# models=
model_names=['LogisticRegression','RandomForest','Adaboost','Bagging',
                                'Naive-Bayes','KNN','Decistion_Tree',
                                'bagged_LR',"adaboost_rf","adaboost_dtc","adaboost_naive_bayes",
                                "adaboost_logistic_regression","gradient_boosting_trees",
             "adaboost_gbc","xgboost","adaboost_xgbc","light_gbm"]

#getting predictions on full data
pred_on_full_data(Xfull,yfull,test_saperated,models=models_final_for_csv,models_name=model_names)

LogisticRegression
RandomForest
Adaboost
Bagging
Naive-Bayes
KNN
Decistion_Tree
bagged_LR
adaboost_rf
adaboost_dtc
adaboost_naive_bayes
adaboost_logistic_regression
gradient_boosting_trees
adaboost_gbc
